# CLV

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahdi-ebrahimi-per/AI/blob/main/8-%20CLV/CLV.ipynb)




In [2]:
import numpy as np
import pandas as pd
from prettytable import PrettyTable
import matplotlib.pyplot as plt


from persiantools.jdatetime import JalaliDate
import datetime
from dateutil.relativedelta import relativedelta


# from sklearn.cluster import KMeans
# from sklearn.preprocessing import scale

# import seaborn as sns

from colorama import Fore

### Load Transaction CSV
    - اطلاعات مربوط به تراکنش های مربوط به ترمینال هاست

In [3]:
# Load Data Frame
Trans_df = pd.read_csv("TRANSACTION.csv", encoding="utf-8-sig")
Trans_df = Trans_df.iloc[:, :10] #ignore last 2 columns
# print(Trans_df.head())


# Unique Value
unique_counts = PrettyTable()
unique_counts.field_names = ["Feature", "Unique Value"]

for column in Trans_df.columns:
    count = Trans_df[column].nunique()
    unique_counts.add_row([column, count])


print(unique_counts)    


# NaN Values
NaN_include_flag = False
for f in list(Trans_df.isna().sum()):
    if f > 0:
        NaN_include_flag = True
        break

if NaN_include_flag:
    print(Fore.RED + "DataFrame Includes NaN Values 👎🏼" + Fore.RESET)

else:
    print(Fore.GREEN + "DataFrame Haven't any NaN Values 👍🏼" + Fore.RESET)


+---------------+--------------+
|    Feature    | Unique Value |
+---------------+--------------+
|  TERMINAL_NO  |      2       |
|    TRANDATE   |      1       |
|  RECENCYDATE  |      2       |
|  TRANNUMDAYS  |      2       |
|   FREQUENCY   |      2       |
|     AMOUNT    |      2       |
|       TC      |      2       |
| TRAN_PURCHASE |      2       |
|  TRAN_CHARGE  |      2       |
|  TRAN_BALANCE |      1       |
+---------------+--------------+
DataFrame Haven't any NaN Values 👍🏼


| TERMINAL_NO | TRANDATE | RECENCYDATE | TRANNUMDAYS | FREQUENCY | AMOUNT | TC | TRAN_PURCHASE | TRAN_CHARGE | TRAN_BALANCE |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| شماره ترمینال | تاریخ | آخرین خرید | تعداد روز هایی که در ماه تراکنش داشته | تعداد تراکنش | مبلغ کل تراکنش | کارمزد | تراکنش خرید | تراکنش شارژ | تراکنش دریافت موجودی |

### Load Merchant CSV
    -  ها است PSP این دیتاست مربوط به اطلاعات  

In [4]:
# Load Data Frame
Merch_df = pd.read_csv("MERCHANT.csv", encoding="utf-8-sig")
# print(df.head())


# Unique Value
unique_counts = PrettyTable()
unique_counts.field_names = ["Feature", "Unique Value"]

for column in Merch_df.columns:
    count = Merch_df[column].nunique()
    unique_counts.add_row([column, count])


print(unique_counts)    


# NaN Values
NaN_include_flag = False
for f in list(Merch_df.isna().sum()):
    if f > 0:
        NaN_include_flag = True
        break

if NaN_include_flag:
    print(Fore.RED + "DataFrame Includes NaN Values 👎🏼" + Fore.RESET)

else:
    print(Fore.GREEN + "DataFrame Haven't any NaN Values 👍🏼" + Fore.RESET)


+-----------------+--------------+
|     Feature     | Unique Value |
+-----------------+--------------+
|   TERMINAL_NO   |      2       |
| MAININSTALLDATE |      2       |
|  DISABLED_DATE  |      2       |
|  CATEGORY_NAME  |      2       |
|   CATEGORYCODE  |      2       |
|   PROVINCENAME  |      2       |
|   PROVINCECODE  |      2       |
|     CITYNAME    |      2       |
|     CITYCODE    |      2       |
|   PROJECTNAME   |      1       |
|   PROJECTCODE   |      1       |
|    AGENTNAME    |      2       |
|    AGENTCODE    |      2       |
|  TERMINALSTATUS |      1       |
|     POSTYPE     |      2       |
|     POSMODEL    |      2       |
|     POSBRAND    |      2       |
| WORKFLOWCAPTION |      1       |
|       TEL1      |      2       |
|   FIRSTNAMEFA   |      2       |
|    LASTNAMEFA   |      2       |
|  OFFICEADDRESS  |      2       |
|      MOBILE     |      2       |
+-----------------+--------------+
DataFrame Haven't any NaN Values 👍🏼


### Join two DataFrame according to "Merchant"

In [5]:

DataFrame = pd.concat([Merch_df, Trans_df], axis=1).reindex(Merch_df.index)


print(DataFrame.isna().sum())




TERMINAL_NO        0
MAININSTALLDATE    0
DISABLED_DATE      0
CATEGORY_NAME      0
CATEGORYCODE       0
PROVINCENAME       0
PROVINCECODE       0
CITYNAME           0
CITYCODE           0
PROJECTNAME        0
PROJECTCODE        0
AGENTNAME          0
AGENTCODE          0
TERMINALSTATUS     0
POSTYPE            0
POSMODEL           0
POSBRAND           0
WORKFLOWCAPTION    0
TEL1               0
FIRSTNAMEFA        0
LASTNAMEFA         0
OFFICEADDRESS      0
MOBILE             0
TERMINAL_NO        0
TRANDATE           0
RECENCYDATE        0
TRANNUMDAYS        0
FREQUENCY          0
AMOUNT             0
TC                 0
TRAN_PURCHASE      0
TRAN_CHARGE        0
TRAN_BALANCE       0
dtype: int64


### Seprate Date into two category 
    - two category with same len

In [6]:


def seprate_time(DataFrame,TimeStepMonth_Seperator=3 ,time_ColumnName="TRANDATE" ,index_ColumnName="TERMINAL_NO"):
    
    Time_df = DataFrame[time_ColumnName]
    
    
    # convert Jalalian to Gregorian
    df_jalaliToGregorian = []
    df_TerminalNumber = []
    
    # print(DataFrame)
    
    for index, row in DataFrame.iterrows():
        # print(int(row["TRANDATE"]))
        gregorian_date = JalaliDate(int(str(int(row["TRANDATE"]))[:4]), int(str(int(row["TRANDATE"]))[4:]),1).to_gregorian()
        df_TerminalNumber.append(int(row["TERMINAL_NO"]))
        df_jalaliToGregorian.append(gregorian_date)


    # print(df_TerminalNumber)
    # print(df_jalaliToGregorian)

    # Delete ME
    df_jalaliToGregorian.append(datetime.date(2000,10,1))
    df_jalaliToGregorian.append(datetime.date(2030,10,30))
    # to here |

    # print(df_jalaliToGregorian) 


    # Find Min and Max of Dates    
    date_min_max = pd.DataFrame(df_jalaliToGregorian).agg(["min", "max"])

    
    # print(date_min_max[0])
    
    
    # Seprate
    
    def time_plus_month(GivenDate, Months=TimeStepMonth_Seperator):
        date_format = '%Y/%m/%d'
        dtObj = datetime.datetime.strptime(GivenDate, date_format)
        future_date = dtObj + relativedelta(months=Months)
        return future_date

    middle = time_plus_month(str(date_min_max[0]['min']).replace("-","/"))
    
    Time_Category_Upper = []
    Time_Category_Under = [] 
    
    
    for date in df_jalaliToGregorian :
        if date > middle.date():
            Time_Category_Upper.append(date)
        else:
            Time_Category_Under.append(date)
    
    print(Time_Category_Upper) 
    print(Time_Category_Under)
    
    
    
    # given_date = '2021/1/21'
    # date_format = '%Y/%m/%d'
    # print('Give Date: ', given_date)
    # dtObj = datetime.datetime.strptime(given_date, date_format)
    # # Add 20 months to a given datetime object
    # n = 3
    # future_date = dtObj + relativedelta(months=n)
    # print(future_date)    
        
        
        
    # # Get Average of Min and Max
    # df = pd.DataFrame({'a': [date_min_max[0]['min']],
    #                    'b': [date_min_max[0]['max']]})
 
    # # make time data into Timestamp format
    # def try_totime(t):
    #     try:
    #         return pd.Timestamp(t)
    #     except:
    #         return np.nan
        
    # df['start'] = df.a.map(try_totime)
    # df['end'] = df.b.map(try_totime)
    
    # # generate midpoint time
    # # harder than it would seem...
    # df['Average_time'] = df.start + (df.end - df.start)/2
    
    # # print(df["Average_time"][0])
    
    
    # # Seprate to two part (Upper than Average and Under Average)
    # Time_Category_Upper = [] 
    # Time_Category_Under = [] 
    
    # for date in df_jalaliToGregorian :
    #     if date > df["Average_time"][0]:
    #         Time_Category_Upper.append(date)
    #     else:
    #         Time_Category_Under.append(date)
    
    # print(Time_Category_Upper) 
    # print(Time_Category_Under)
    
    
# مشکل
    # اضافه کردن مشخصه ی شماره ترمینال به تاریخ
    
    
print(seprate_time(Trans_df, 4))


[datetime.date(2021, 12, 22), datetime.date(2021, 12, 22), datetime.date(2030, 10, 30)]
[datetime.date(2000, 10, 1)]
None


### Make Unique Terminal number per sumOfFeatures
    - use Maximum for RECENCYDATE

In [7]:
Trans_df.head()

,TERMINAL_NO,TRANDATE,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TC,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE
0,4133705,140010,14001030,25,131,469340000,262540.0,131,0,0
1,41294305,140010,14001029,21,46,1943933000,84460.0,41,5,0


In [14]:
TerminalNo_Unique = list(Trans_df.TERMINAL_NO.unique())

Max_RECENCYDATE = []
Sum_TRANNUMDAYS = []
Sum_FREQUENCY = []
Sum_AMOUNT = []
Sum_TC = []
Sum_TRAN_PURCHASE = []
Sum_TRAN_CHARGE = []
Sum_TRAN_BALANCE = []

# Max_RECENCYDATE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["RECENCYDATE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Max_RECENCYDATE.append(max(Listed))

# print(Max_RECENCYDATE)

# Sum_TRANNUMDAYS
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRANNUMDAYS"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRANNUMDAYS.append(sum(Listed))


# Sum_FREQUENCY
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["FREQUENCY"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_FREQUENCY.append(sum(Listed))



# Sum_AMOUNT
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["AMOUNT"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_AMOUNT.append(sum(Listed))



# Sum_TC
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TC"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TC.append(sum(Listed))



# Sum_TRAN_PURCHASE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_PURCHASE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_PURCHASE.append(sum(Listed))



# Sum_TRAN_CHARGE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_CHARGE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_CHARGE.append(sum(Listed))





Feature Extraction
Feature Selection
Encoding
    - str to categirical
    - one hot encoding